In [2]:
from gurobipy import *
import gurobipy as gp

# type declaration
from typing import Dict, List

# %load_ext nb_black
%load_ext lab_black

## Question 1

In [3]:
toys: List[str] = ["Space", "Zapper", "Big", "Soaker"]

resources: List[str] = ["plastic", "time"]

r: Dict[tuple, float] = {
    ("plastic", "Space"): 2,
    ("plastic", "Zapper"): 1,
    ("plastic", "Big"): 3,
    ("plastic", "Soaker"): 4,
    ("time", "Space"): 3,
    ("time", "Zapper"): 4,
    ("time", "Big"): 5,
    ("time", "Soaker"): 6,
}

availability: Dict[str, int] = {"plastic": 3000, "time": 6000}

profit: Dict[str, int] = {"Space": 16, "Zapper": 15, "Big": 20, "Soaker": 22}

In [4]:
# model
model = gp.Model()
x = model.addVars(toys, obj=profit, ub=1000, lb=100)

model.ModelSense = gp.GRB.MAXIMIZE

resourceCons = model.addConstrs(
    quicksum(r[(i, j)] * x[j] for j in toys) <= availability[i] for i in resources
)
model.optimize()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04


In [5]:
for toy in range(len(toys)):
    print("%s-toys to produce:" % toys[toy], model.x[toy])

Space-toys to produce: 860.0
Zapper-toys to produce: 580.0
Big-toys to produce: 100.0
Soaker-toys to produce: 100.0


#### Using forloops

In [6]:
products: List[str] = ["Space", "Zapper", "Big", "Soaker"]

resources: List[str] = ["plastic", "time"]

resUse: Dict[tuple, float] = {
    ("plastic", "Space"): 2,
    ("plastic", "Zapper"): 1,
    ("plastic", "Big"): 3,
    ("plastic", "Soaker"): 4,
    ("time", "Space"): 3,
    ("time", "Zapper"): 4,
    ("time", "Big"): 5,
    ("time", "Soaker"): 6,
}

resAvail: Dict[str, int] = {"plastic": 3000, "time": 6000}

profit: Dict[str, int] = {"Space": 16, "Zapper": 15, "Big": 20, "Soaker": 22}

In [7]:
model = gp.Model()
pvars = {}


for p in products:
    """
    For each product set the same constrains of upper lower bound and set the objective
    Same as: x = model.addVars(toys, obj=profit, ub=1000, lb=100)
    """
    pvars[p] = model.addVar(lb=100, ub=1000, obj=profit[p], name=p)
model.ModelSense = gp.GRB.MAXIMIZE

for r in resources:
    """
    Here for each resource: loop through each product
    resourceCons = model.addConstrs(
    quicksum(r[(i, j)] * x[j] for j in toys) <= availability[i] for i in resources)

    """
    lhs = 0
    for p in products:
        """
        For each ressoruce there is the same availablity in resoruce: so match each resource to each product!
        --> quicksum is just the sum of all combinations of resources and products here!
        Here you calculate how much of a certain resource you are using
        """
        lhs += resUse[r, p] * pvars[p]
    model.addConstr(lhs <= resAvail[r])

print("\n")
model.optimize()
print("objective value is:", model.ObjVal)

print("\n")
for product in range(len(products)):
    print("%s-toys to produce:" % products[product], model.x[product])
None



Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 4 columns and 8 nonzeros
Model fingerprint: 0xe7d9c8b2
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [2e+01, 2e+01]
  Bounds range     [1e+02, 1e+03]
  RHS range        [3e+03, 6e+03]
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7300000e+04   5.125000e+02   0.000000e+00      0s
       2    2.6660000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.666000000e+04
objective value is: 26660.0


Space-toys to produce: 860.0
Zapper-toys to produce: 580.0
Big-toys to produce: 100.0
Soaker-toys to produce: 100.0


## Question 2


In [ ]:
makeCosts=[6,13,20]
buyACosts=[12,15,21]
buyBCosts=[11,16,23]
laborUse=[1,2,3]
demands=[100,80,70]
aProdLimit=60
laborAvail=200
n=3 #vars num
model=gp.Model()
M=model.addVars(n,name="make",obj=makeCosts)
A=model.addVars(n,name="BuyA",obj=buyACosts)
B=model.addVars(n,name="BuyB",obj=buyBCosts)
#by default minimization
model.addConstrs(M[i]+A[i]+B[i]==demands[i] for i in range(n)) #sattisfy demands
model.addConstr(gp.quicksum(laborUse[i]*M[i] for i in range(n))<=laborAvail)
model.addConstr(gp.quicksum(A[i] for i in range(n))<=aProdLimit)
model.optimize()
if not model.status == gp.GRB.OPTIMAL:
    print("something went wrong")
print("optimal value",model.objval)
model.printAttr("X")

Essentially: the process is as follows:
- You need to minimize the ALL costs of you as a provider but also the subcontractors
- a good way to imagine the DECISIONS (what to minimize) is to not think of you as a provider but just: you have 3 different providers 


    

In [21]:
makeCosts = [6, 13, 20]
buyACosts = [12, 15, 21]
buyBCosts = [11, 16, 23]
laborUse = [1, 2, 3]
demands = [100, 80, 70]
aProdLimit = 60
laborAvail = 200
n = 3  # number of vars

In [62]:
# init the model
model = gp.Model()

# now init each production line
#  you need each on its own becasue M (make) underlies different constraints
"""
IMPORTANT: If you have different constraints applying to different "line" then 
create different vairables onto which to apply these 
EG here only M has the labour hour constraints; so this one is relevant (and n just stands for the number of 
products in total; this may also be RELEVANT!)

--> eg, for certain lines, the production line n might differ as well!!! (so eg A does not product prod 
2 or so; then this way you can constrain it)
"""
# THIS WAY YOU CAN ALSO DO M[i] for instance!!! to address each specifically!!

M = model.addVars(n, name="make", obj=makeCosts)
A = model.addVars(n, name="BuyA", obj=buyACosts)
B = model.addVars(n, name="BuyB", obj=buyBCosts)


# first constraint of working hours for M only
lhs = 0
for i in range(n):
    """
    This for loop shoud accomplish the same as:
    model.addConstr(gp.quicksum(laborUse[i] * M[i] for i in range(n)) <= laborAvail)

    IMPORTANT: Gurobi does not like it if you do operations in the addConstr function;
    so dont do model.addConstr(laborUse[i] * M[i] <= laborAvail) --> use the lhs to assign to!!!

    Important as well; you could technically just loop thorugh adding the constraints as well here
    (so indenting the model.addConstr(lhs <= laborAvail) as well!)
    """
    lhs += laborUse[i] * M[i]

# important: addConstr --> ONE CONSTRAINT!!!!!!!!
model.addConstr(lhs <= laborAvail)


# second constraint of A only being able to use 60 in TOTAL for all products in question
for i in range(n):
    """
    this loop should accomplish about the same as:
    model.addConstr(gp.quicksum(A[i] for i in range(n)) <= aProdLimit)

    --> The main point is that here for instance we could impose specific restricions
    One example is that all but the last product ccan be produced 60 times each but the last one is 50
    and you can do this by manipulating the loop accordingly range(n-1) (would leave out the last one)
    """
    model.addConstr(A[i] <= aProdLimit)


# finally the demand constraint
for i in range(n):
    lhs = 0
    for production_location in [M, A, B]:
        """
        For each product (1,2,3) there is the same constraint for each production location (M, A, B)
        --> Here again: use addConstr and not addConstrs; the reason is that you have constraints
        for product 1 all --> So it is not multiple constraints at once!


        model.addConstrs(M[i] + A[i] + B[i] == demands[i] for i in range(n))
        """
        lhs += production_location[i]
    model.addConstr(lhs == demands[i])


# be explicit
model.ModelSense = gp.GRB.MINIMIZE


model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 9 columns and 15 nonzeros
Model fingerprint: 0x7ab2ab47
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolved: 3 rows, 7 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0400000e+03   1.769142e+01   0.000000e+00      0s
       2    3.1900000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.190000000e+03


In [63]:
# # minimize by default --> only use make variables becasue this is the one you are minimizing
# model.addConstr(gp.quicksum(laborUse[i] * M[i] for i in range(n)) <= laborAvail)
# model.addConstr(gp.quicksum(A[i] for i in range(n)) <= aProdLimit)

# # these are multiple constraints now
# model.addConstrs(M[i] + A[i] + B[i] == demands[i] for i in range(n))


model.optimize()
print("optimal value", model.objval)
model.printAttr("X")

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 9 columns and 15 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.190000000e+03
optimal value 3190.0

    Variable            X 
-------------------------
     make[0]          100 
     make[1]           50 
     BuyA[1]           30 
     BuyA[2]           60 
     BuyB[2]           10 


## Question 3


## Question 4